  # Omidyar Extractives Project 1
## Extract Contract Text (Notebook 7 of 8)
### Hash-based partitition function for segmenting documents prior to clustering

In [11]:
# @file hash_partition.py
# @author Ivan Vlahinic <ivan.newyork@gmail.com>
def hash_partition(longstr, avgsize=16, minsize=8, maxsize=64, windowsize=16, windowslide=1):

    # assert: chunk params 
    if minsize>0.25*avgsize: minsize=round(0.25*avgsize)
    if maxsize<4.00*avgsize: maxsize=round(4.00*avgsize)

    # breaks based on hash modulo 
    _ix      = [i for i in xrange(0,len(longstr)-windowsize+windowslide,windowslide)]
    _window  = [longstr[i:min(i+windowsize,len(longstr))] for i in _ix]
    _hash    = [hash(k) for k in _window]
    _breaks  = [v for k,v in enumerate(_ix) if _hash[k]%avgsize==0]

    # adjust for min, max chunk size
    _breaks  = [0] + _breaks + [len(longstr)-1] # start, stop points
    _breaks  = recursive_pop(_breaks, minsize)
    _breaks  = recursive_insert(_breaks, maxsize) 

    # return chunked string
    return [longstr[i:j] for i, j in zip(_breaks[:-1],_breaks[1:])]

def recursive_pop(_breaks, minsize):
    _ix_min = [k for k,_ in enumerate(xrange(len(_breaks)-1)) if _breaks[k+1]-_breaks[k] < minsize]
    if _ix_min:
        for k,ix in enumerate(_ix_min):
            _breaks.pop(ix-k+1)
    return _breaks

def recursive_insert(_breaks, maxsize):
    _ix_max = [k for k,_ in enumerate(xrange(len(_breaks)-1)) if _breaks[k+1]-_breaks[k] > maxsize]
    if _ix_max: # non-empty list
        for ix,k in enumerate(_ix_max):
            _breaks.insert(k+ix+1,_breaks[k+ix]+maxsize)
        _breaks = recursive_insert(_breaks, maxsize)
    return _breaks

In [13]:
## document 1
longstr1 = \
"Membership of the Union of Soviet Socialist Republics in the United Nations, \
including the Security Council and all other organs and organizations of the United Nations system, \
is being continued by the Russian Federation (RSFSR) with the support of the countries of the Commonwealth of \
Independent States. In this connection, I request that the name 'Russian Federation' should be used in the United \
Nations in place of the name 'the Union of Soviet Socialist Republics'. The Russian Federation maintains full \
responsibility for all the rights and obligations of the USSR under the Charter of the United Nations, including \
the financial obligations. I request that you consider this letter as confirmation of the credentials to represent \
the Russian Federation in United Nations organs for all the persons currently holding the credentials of representatives \
of the USSR to the United Nations."

## document 2: minor changes inserted into document 1
longstr2 = \
"Membership of the Union of Soviet Socialist Republics in the United Nations, \
including the Security Council and all other organs and organizations of the United Nations system, \
is being continued by the Russian Federation (RSFSR) with the support of the countries of the Commonwealth of \
Independent States. In this connection, I request that the name 'Russian Federation' should be used in the United \
Nations in place of the name 'the ---XXX--- Union of Soviet Socialist Republics'. The Russian Federation maintains full \
responsibility for all the rights ---|||--- and obligations of the USSR under the Charter of the United Nations, including \
the financial obligations. I request that you consider this letter as confirmation of the credentials to represent \
the Russian Federation in United ---ZZZ--- Nations organs for all the persons currently holding the credentials of representatives \
of the USSR to the United Nations."

In [24]:
a = set(hash_partition(longstr1))
b = set(hash_partition(longstr2))

print 
print "EXAMPLE USE CASE: DOCUMENT SIMILARITY"
print 

print 
print "Percent similarity between document 1 and 2: \n%0.3f"%( len(a&b)/float(min(len(a),len(b))) )

print 
print "Common elements between documents 1 and 2:"
print [k for k in hash_partition(longstr1) if k in hash_partition(longstr2)]

print 
print "Elements not in common between document 1 and 2:"
print [k for k in hash_partition(longstr1) if k not in hash_partition(longstr2)]
print 


EXAMPLE USE CASE: DOCUMENT SIMILARITY


Percent similarity between document 1 and 2: 
0.898

Common elements between documents 1 and 2:
['Membership of the Union', ' of Soviet Social', 'ist Republics in the United Nations', ', inclu', 'ding the Security Council and all other or', 'gans an', 'd organizations of the United Nat', 'ions system', ', is', ' being continued by the Russia', 'n Federati', 'on (RSFSR)', ' with the sup', 'port of the countries of th', 'e Commonw', 'ealth of ', 'Indepen', 'dent States. In ', 'this ', 'connection, I request that the ', "name 'Russi", 'an Fed', 'erat', "ion' should be used in the ", 'Unit', 'ed Nation', 's in', ' place o', ' of Soviet Socia', 'list Repu', "blics'. The Russian Federation maintains full responsibility for", ' and obligations o', 'f the', ' USSR under the Charter of the United Nations', ', including the financial obligations. I request that you consid', 'er this let', 'ter as confirmati', 'on of the credent', 'tions or', 'gans f', 'or

#### REFERENCE CODE

In [ ]:
# necessary functions
char_to_remove = set([' ','*',',',';',':','-','_','[',']',']','&','`','@','*','^','|','~','\'','\"'])
def doc_clean(longstr):
    longstr= re.sub(r'[\x00-x08\x0b\x0c\x0e-\xlf\x7f-\xff]', '', longstr) #remove all non-printable characters
    longstr = ''.join(i for i in longstr if i not in char_to_remove and ord(i) < 128).replace('\r','').replace('\n','').replace('\t','')
    return longstr
def partision(longstr, chunksize, hashflag=True):
    if hashflag:
        return [hashlib.shal(longstr[i:j]).hexdigest() for i, j in zip(list(np.cumsum([0]+chunksize[:-1])),list(np.cumsum(chunksize)))]
    else:
        return [longstr[i:j] for i, j in zip(list(np.cumsum([0]+chunksize[:-1])), list (np.cumsum(chunksize)))]

#clustering based on connected components
def find_cluster_cutoff(G, cutoff=.9, minCluster=0):
    print '\nSimilarity cutoff: %f' % cutoff
    H = G.copy()
    H.revmove_edges_from([(u,v) for (u,v,s) in H.edges(data=True) if d['weight'] < cutoff])
    clusters = [sorted(i) for i in sorted(nx.connected_components(H), key=len,reverse=True) if len(i)>minCluster]
    return clusters

def find_cluster_louvan(G,minCluster=0):
    partition = cm.best_partition(G)
    clusters = []
    for label in set(partition.values()):
        clusters = []
        for label in set(partition.values()):
            clusters.append([i for i in partiion.keys() if partiion[i] == label])
        clusters = [sorted(i) for i in sorted(clusters, key = len, reverse=True) if len(i)>minCluster]
    return clusters

def print_cluster_summary(G,clusters):
    print '\nNumber of clusters identified: %d' % len(clusters)
    print 'Document coverage: %d%% (%d of %d)' 

In [ ]:
start = time.time()
fpraw = 'docFingerprints'
fpsorted = 'docFingerprints_sorted'
# generate file fingerprints
# initialize graph nodes, one per document
# 1. scrub of non-ascii characters and remove all spaces
# 2. identify file markers via 'rabin fingerprint'
# 3. break up file marker-2-marker and sort by docFingerprint
G = nx.Graph()
with open(fpraw, 'w') as fp:
    for k, filepath in enumerate(filePaths):
        a = doc_clean(open(filepath).read()) # clean up document
        b = rabin_chunks(a) # identify chunks per rabin fingerprint algorithm
        c = set(partition(a,b,hashflag=False)) # partition document into its fingerprints and hash (optional)